In [71]:
import requests
import pandas as pd
import json
from dateutil import parser

In [12]:
API_KEY = "1855f9da622fd5008d8ccee9d3cb00c7-90fc8756c93f3c9ac58184f8fb734a90"
ACCOUNT_ID = "101-001-26467877-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [13]:
session = requests.Session()

In [14]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [22]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [27]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [28]:
response = session.get(url, params = None, data = None, headers = None)

In [29]:
response.status_code

200

In [30]:
data = response.json()

In [33]:
instruments_list = data['instruments']

In [34]:
len(instruments_list)

68

In [37]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [39]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 
         'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [46]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i}

In [48]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [49]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent = 2))

In [106]:
#Function which actually fetches the candle data from OANDA API
def fetch_candles(pair_name, count = 10, granularity = "H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params = params, data = None, headers = None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
#Function which gets candle data and converts it to a dataframe
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    #mid_c, mid_o, bid_h, etc..
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df
#Function to save the file of Candle Data
def create_data_file(pair_name, count = 10, granularity = "H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No Candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pk1")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [108]:
code, data = fetch_candles("EUR_USD", count=10, granularity = "H4")
candles_df = get_candles_df(data)

In [109]:
create_data_file("EUR_USD", count=10, granularity = "H4")

EUR_USD H4 10 candles, 2023-07-27 05:00:00+00:00 2023-07-28 17:00:00+00:00


In [112]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [117]:
#Looping through all pairs of currencies in our_curr
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4000, granularity=g)

EUR_USD H1 4000 candles, 2022-12-07 05:00:00+00:00 2023-07-28 20:00:00+00:00
EUR_USD H4 4000 candles, 2021-01-06 06:00:00+00:00 2023-07-28 17:00:00+00:00
EUR_GBP H1 4000 candles, 2022-12-07 05:00:00+00:00 2023-07-28 20:00:00+00:00
EUR_GBP H4 4000 candles, 2021-01-06 06:00:00+00:00 2023-07-28 17:00:00+00:00
EUR_JPY H1 4000 candles, 2022-12-07 05:00:00+00:00 2023-07-28 20:00:00+00:00
EUR_JPY H4 4000 candles, 2021-01-05 22:00:00+00:00 2023-07-28 17:00:00+00:00
EUR_CHF H1 4000 candles, 2022-12-07 05:00:00+00:00 2023-07-28 20:00:00+00:00
EUR_CHF H4 4000 candles, 2021-01-06 06:00:00+00:00 2023-07-28 17:00:00+00:00
EUR_NZD H1 4000 candles, 2022-12-07 05:00:00+00:00 2023-07-28 20:00:00+00:00
EUR_NZD H4 4000 candles, 2021-01-06 06:00:00+00:00 2023-07-28 17:00:00+00:00
EUR_CAD H1 4000 candles, 2022-12-07 05:00:00+00:00 2023-07-28 20:00:00+00:00
EUR_CAD H4 4000 candles, 2021-01-06 06:00:00+00:00 2023-07-28 17:00:00+00:00
EUR_AUD H1 4000 candles, 2022-12-07 04:00:00+00:00 2023-07-28 20:00:00+00:00